In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [3]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-02 12:02:23.664339: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-02 12:02:23.664510: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
train = train.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
val = val.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
test = test.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))

model = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    pooling='avg',
    classes=6,
    classifier_activation="softmax"
)

model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [4]:
# train = train.map(lambda x, y: (tf.keras.applications.xception.preprocess_input(x), y))
# val = val.map(lambda x, y: (tf.keras.applications.xception.preprocess_input(x), y))
# test = test.map(lambda x, y: (tf.keras.applications.xception.preprocess_input(x), y))

# model = tf.keras.applications.Xception(
#     include_top=True,
#     weights=None,
#     input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
#     pooling='avg',
#     classes=6,
#     classifier_activation="softmax"
# )

# model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 49, 49, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 49, 49, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                           

In [5]:
model.compile(optimizer=keras.optimizers.RMSprop(0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks_list = [
    # keras.callbacks.EarlyStopping(
    #     monitor="val_accuracy",
    #     patience=8
    # ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/sota_1.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history = model.fit(
  train.cache(),
  epochs=20,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

2022-08-02 12:02:49.795677: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20


2022-08-02 12:02:50.471934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.3463 - accuracy: 0.4929

2022-08-02 12:03:46.547300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 60s 563ms/step - loss: 1.3463 - accuracy: 0.4929 - val_loss: 0.7615 - val_accuracy: 0.7278
Epoch 2/20
105/105 [==============================] - 64s 611ms/step - loss: 0.6042 - accuracy: 0.7807 - val_loss: 0.5468 - val_accuracy: 0.8264
Epoch 3/20
105/105 [==============================] - 66s 626ms/step - loss: 0.3432 - accuracy: 0.8789 - val_loss: 0.5358 - val_accuracy: 0.8458
Epoch 4/20
105/105 [==============================] - 67s 637ms/step - loss: 0.2300 - accuracy: 0.9208 - val_loss: 0.4021 - val_accuracy: 0.8806
Epoch 5/20
105/105 [==============================] - 70s 667ms/step - loss: 0.1688 - accuracy: 0.9438 - val_loss: 0.3900 - val_accuracy: 0.9153
Epoch 6/20
105/105 [==============================] - 71s 674ms/step - loss: 0.1386 - accuracy: 0.9619 - val_loss: 0.2230 - val_accuracy: 0.9319
Epoch 7/20
105/105 [==============================] - 69s 656ms/step - loss: 0.1135 - accuracy: 0.9652 - val_loss: 0.3187 - val_accuracy: 0.9

In [6]:
model.evaluate(test)

load_model = keras.models.load_model("models/sota_1.h5")
load_model.evaluate(test)

23/23 [==============================] - 3s 121ms/step - loss: 0.4903 - accuracy: 0.9528


2022-08-02 12:25:58.701948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 3s 135ms/step - loss: 0.4903 - accuracy: 0.9528


[0.4902770221233368, 0.9527778029441833]